In [1]:
import numpy as np
import scipy
import pandas as pd
import pickle
import glob
import seaborn as sns
import matplotlib.pyplot as plt
import networkx as nx
import time

from math import nan
from datetime import datetime

import csv

from tqdm import tqdm

import vaex

import matplotlib.pyplot as plt
plt.rcParams.update({'font.size': 13})

In [2]:
d = glob.glob('/scratch/Debbie_Miori/*')
d

['/scratch/Debbie_Miori/CLOSE_20000128_20190701.csv',
 '/scratch/Debbie_Miori/imbalances_cusip',
 '/scratch/Debbie_Miori/holdings_t',
 '/scratch/Debbie_Miori/stocks.csv',
 '/scratch/Debbie_Miori/temporal_filings.pickle',
 '/scratch/Debbie_Miori/holdings_t (copy)',
 '/scratch/Debbie_Miori/13F2022.csv',
 '/scratch/Debbie_Miori/extracted_imbalances_by_cik.pickle',
 '/scratch/Debbie_Miori/adj_close_prices',
 '/scratch/Debbie_Miori/PROD.zip',
 '/scratch/Debbie_Miori/imbalances_cusip (copy)',
 '/scratch/Debbie_Miori/holdings_by_cik.pickle',
 '/scratch/Debbie_Miori/frequency_filing_clean.pickle',
 '/scratch/Debbie_Miori/stocks_data',
 '/scratch/Debbie_Miori/NPORT2022.csv',
 '/scratch/Debbie_Miori/Old',
 '/scratch/Debbie_Miori/NLP-project',
 '/scratch/Debbie_Miori/Oct2021',
 '/scratch/Debbie_Miori/big_dataset_dates.pickle']

### Upload the file with all raw information on the 13F filings

In [3]:
df_13f = vaex.from_csv('/scratch/Debbie_Miori/13F2022.csv', copy_index=False, error_bad_lines=False)
df_13f

/home/miori/.local/lib/python3.8/site-packages/vaex/__init__.py:526: FutureWarning: The error_bad_lines argument has been deprecated and will be removed in a future version.


  return _from_csv_read(filename_or_buffer=filename_or_buffer, copy_index=copy_index,


#,reg_cik,reg_name,reg_lei,series_id,series_name,series_lei,period,as_of_date,form,security_type,security_class,payoff,name,lei,issuer,isin,cusip,sedol,valor,ticker,quantity,valuation,valuation_currency,allocation,file,line
0,1497637,Accuvest Global Advisors,nan,nan,nan,nan,2017-12-31,2018-01-30,13F-HR,shares,COM,Long,Blackrock Core Bond Trust,nan,09249e,nan,09249e101,nan,nan,nan,24065.0,338.0,USD,nan,1497637/13F-HR/0001497637-18-000001.txt,110
1,1497637,Accuvest Global Advisors,nan,nan,nan,nan,2017-12-31,2018-01-30,13F-HR,shares,COM,Long,Eaton Vance Ltd Duration Incom,nan,27828h,nan,27828h105,nan,nan,nan,14000.0,191.0,USD,nan,1497637/13F-HR/0001497637-18-000001.txt,126
2,1497637,Accuvest Global Advisors,nan,nan,nan,nan,2017-12-31,2018-01-30,13F-HR,shares,COM,Long,FIRST TRUST SENIOR FLOATING,nan,33733U,nan,33733U108,nan,nan,nan,68251.0,880.0,USD,nan,1497637/13F-HR/0001497637-18-000001.txt,142
3,1497637,Accuvest Global Advisors,nan,nan,nan,nan,2017-12-31,2018-01-30,13F-HR,shares,COM,Long,High Yield L/S ETF - First Tru,nan,33738d,nan,33738d408,nan,nan,nan,17050.0,828.0,USD,nan,1497637/13F-HR/0001497637-18-000001.txt,158
4,1497637,Accuvest Global Advisors,nan,nan,nan,nan,2017-12-31,2018-01-30,13F-HR,shares,COM,Long,INVESCO MUNICIPAL TRUST,nan,46131J,nan,46131J103,nan,nan,nan,30900.0,386.0,USD,nan,1497637/13F-HR/0001497637-18-000001.txt,174
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
"65,442,764",1730962,"ODonnell Financial Services, LLC",nan,nan,nan,nan,2021-09-30,2021-10-29,13F-HR,shares,COM,Long,TESLA INC,nan,88160R,nan,88160R101,nan,nan,nan,600.0,465.0,USD,nan,1730962/13F-HR/0001085146-21-002940.txt,815
"65,442,765",1730962,"ODonnell Financial Services, LLC",nan,nan,nan,nan,2021-09-30,2021-10-29,13F-HR,shares,COM,Long,UNITEDHEALTH GROUP INC,nan,91324P,nan,91324P102,nan,nan,nan,1753.0,685.0,USD,nan,1730962/13F-HR/0001085146-21-002940.txt,831
"65,442,766",1730962,"ODonnell Financial Services, LLC",nan,nan,nan,nan,2021-09-30,2021-10-29,13F-HR,shares,TOTAL BND MRKT,Long,VANGUARD BD INDEX FDS,nan,921937,nan,921937835,nan,nan,nan,2406.0,206.0,USD,nan,1730962/13F-HR/0001085146-21-002940.txt,847
"65,442,767",1730962,"ODonnell Financial Services, LLC",nan,nan,nan,nan,2021-09-30,2021-10-29,13F-HR,shares,GROWTH ETF,Long,VANGUARD INDEX FDS,nan,922908,nan,922908736,nan,nan,nan,695.0,202.0,USD,nan,1730962/13F-HR/0001085146-21-002940.txt,863


### Pre-processing and first look

In [4]:
df = df_13f[df_13f.quantity > 0]

df = df.drop(['reg_lei', 'series_id', 'series_name',
             'as_of_date', 'payoff', 'lei', 'isin',
             'file', 'line', 'series_lei', 'sedol', 'valor', 'ticker'
             ], 1)

df = df[df.form == '13F-HR']
df = df[df.security_type == 'shares']

In [5]:
df.head(3)

#,reg_cik,reg_name,period,form,security_type,security_class,name,issuer,cusip,quantity,valuation,valuation_currency,allocation
0,1497637,Accuvest Global Advisors,2017-12-31,13F-HR,shares,COM,Blackrock Core Bond Trust,09249e,09249e101,24065,338,USD,nan
1,1497637,Accuvest Global Advisors,2017-12-31,13F-HR,shares,COM,Eaton Vance Ltd Duration Incom,27828h,27828h105,14000,191,USD,nan
2,1497637,Accuvest Global Advisors,2017-12-31,13F-HR,shares,COM,FIRST TRUST SENIOR FLOATING,33733U,33733U108,68251,880,USD,nan


In [6]:
# Extract all the unique periods available
periods = np.unique(df.period.values)

In [5]:
# Check the interval for which you have good data (or at least a coherent amount)
for p in periods:
    print(p, len(df[df.period==p]))

2006-09-30 178
2006-12-31 13
2007-03-31 17
2007-06-30 114
2007-09-30 133
2007-12-31 171
2008-03-31 127
2008-06-30 171
2008-09-30 164
2008-12-31 598
2009-03-31 990
2009-06-30 1006
2009-09-30 3728
2009-12-31 4123
2010-03-31 4068
2010-06-30 1013
2010-09-30 611
2010-12-31 1979
2011-03-31 1813
2011-06-30 2850
2011-09-30 4457
2011-12-31 5198
2012-03-31 6727
2012-06-30 6860
2012-09-30 4969
2012-12-31 10710
2013-03-31 31510
2013-06-30 1233237
2013-09-30 1256664
2013-12-31 1296609
2014-03-31 1324763
2014-06-30 1341859
2014-09-30 1351553
2014-12-31 1417978
2015-03-31 1437663
2015-06-30 1455082
2015-09-30 1439845
2015-12-31 1480163
2016-03-31 1480824
2016-06-30 1505984
2016-09-30 1496886
2016-12-31 1546231
2017-03-31 1568475
2017-06-30 1568488
2017-09-30 1567898
2017-12-31 1696661
2018-03-31 1718036
2018-06-30 1742753
2018-09-30 1753767
2018-12-31 1752873
2019-03-31 1847602
2019-06-30 1843852
2019-09-30 1814746
2019-12-31 1891649
2020-03-31 1789108
2020-06-30 1793412
2020-09-30 1830363
2020-12-31

In [7]:
sub_periods = periods[27:]
print(len(sub_periods))
print(sub_periods)

34
['2013-06-30' '2013-09-30' '2013-12-31' '2014-03-31' '2014-06-30'
 '2014-09-30' '2014-12-31' '2015-03-31' '2015-06-30' '2015-09-30'
 '2015-12-31' '2016-03-31' '2016-06-30' '2016-09-30' '2016-12-31'
 '2017-03-31' '2017-06-30' '2017-09-30' '2017-12-31' '2018-03-31'
 '2018-06-30' '2018-09-30' '2018-12-31' '2019-03-31' '2019-06-30'
 '2019-09-30' '2019-12-31' '2020-03-31' '2020-06-30' '2020-09-30'
 '2020-12-31' '2021-03-31' '2021-06-30' '2021-09-30']


In [8]:
"""
    For each period p, save the related holdings.
    This will be useful to compute imbalances efficiently
"""

for t in sub_periods:
    print(t)
    curr = df[df.period==t]
    curr_ciks = np.unique(curr.reg_cik.values)  
    cus = pd.DataFrame({'cusips': np.array(curr.cusip.values)}).dropna()
    curr_cusips = np.unique(cus.cusips)
    
    #d = pd.DataFrame(0, index=curr_ciks, columns=curr_cusips)
    
    path = '/scratch/Debbie_Miori/holdings_t/'+str(t)+'.csv'
    #dict_ = {cik:{} for cik in curr_ciks}
    dict_ = {}
    for cik in tqdm(curr_ciks):
        sub = curr[curr.reg_cik==cik]
        
        summed = pd.DataFrame.from_dict({'cusip': sub.cusip.values, 
                                         'quantity': sub.quantity.values}).groupby('cusip').sum()
        #dict_[cik] = dict(zip(list(summed.index), list(summed.quantity)))
        dict_[cik] = summed.to_dict()['quantity']
        
    #for r in tqdm(range(len(curr))):
        #row = curr[r]
        #d.loc[curr[r][0], curr[r][8]] = curr[r][9]
        #dict_[(row[0], row[8])] = row[9]
        
    d = pd.DataFrame.from_dict(dict_, orient='index').fillna(0)
    d.to_csv(path)

2013-06-30


100%|██████████| 3372/3372 [2:52:41<00:00,  3.07s/it]  


2013-09-30


100%|██████████| 3373/3373 [2:51:54<00:00,  3.06s/it]  


2013-12-31


...

100%|██████████| 5869/5869 [4:57:27<00:00,  3.04s/it]  


2021-06-30


100%|██████████| 5874/5874 [4:58:08<00:00,  3.05s/it]  


2021-09-30


100%|██████████| 5857/5857 [4:58:42<00:00,  3.06s/it]  


In [3]:
glob.glob('/scratch/Debbie_Miori/holdings_t/*')

['/scratch/Debbie_Miori/holdings_t/2018-09-30.csv',
 '/scratch/Debbie_Miori/holdings_t/2014-06-30.csv',
 '/scratch/Debbie_Miori/holdings_t/2016-09-30.csv',
 '/scratch/Debbie_Miori/holdings_t/2014-09-30.csv',
 '/scratch/Debbie_Miori/holdings_t/2020-09-30.csv',
 '/scratch/Debbie_Miori/holdings_t/2016-06-30.csv',
 '/scratch/Debbie_Miori/holdings_t/2016-12-31.csv',
 '/scratch/Debbie_Miori/holdings_t/2015-09-30.csv',
 '/scratch/Debbie_Miori/holdings_t/2015-03-31.csv',
 '/scratch/Debbie_Miori/holdings_t/2017-12-31.csv',
 '/scratch/Debbie_Miori/holdings_t/2017-03-31.csv',
 '/scratch/Debbie_Miori/holdings_t/2017-09-30.csv',
 '/scratch/Debbie_Miori/holdings_t/2019-12-31.csv',
 '/scratch/Debbie_Miori/holdings_t/2020-06-30.csv',
 '/scratch/Debbie_Miori/holdings_t/2021-06-30.csv',
 '/scratch/Debbie_Miori/holdings_t/2018-12-31.csv',
 '/scratch/Debbie_Miori/holdings_t/2018-06-30.csv',
 '/scratch/Debbie_Miori/holdings_t/2014-03-31.csv',
 '/scratch/Debbie_Miori/holdings_t/2018-03-31.csv',
 '/scratch/D

In [4]:
check = pd.read_csv('/scratch/Debbie_Miori/holdings_t/2013-06-30.csv')
check

,Unnamed: 0,00206R102,008252108,018490102,020002101,032511107,037833100,053015103,064058100,075896100,...,78469X107,88079F107,92913A100,97653G100,G5960L103,30040W108,97718W108,146282108,30231H100,91059D100
0,2230,584000.0,51700.0,128000.0,355000.0,50000.0,93000.0,200000.0,323775.0,105000.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,3520,63247.0,168485.0,0.0,700.0,1452740.0,1533521.0,0.0,180.0,349.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,5272,6541023.0,168429.0,182441.0,225290.0,284599.0,475448.0,252676.0,448480.0,97168.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,7789,207601.0,0.0,5104.0,122985.0,0.0,83730.0,17755.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,8565,1212432.0,3000.0,25349.0,110384.0,1092025.0,167975.0,208463.0,1056104.0,41182.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3367,1424606,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3368,1537430,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3369,1353326,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3370,1426266,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


### We have extracted holdings at each period p and saved them in different files. Now we have a look at the number of CIKs that we have

In [5]:
import glob
import pandas as pd
from tqdm.notebook import tqdm

In [7]:
ciks = []
for file in tqdm(glob.glob('/scratch/Debbie_Miori/holdings_t/*')):
    ciks += list(pd.read_csv(file).iloc[:, 0])

  0%|          | 0/34 [00:00<?, ?it/s]

In [9]:
import numpy as np
len(np.unique(ciks))

8166

In [10]:
np.unique(ciks)[:10]

array([2230, 3520, 5272, 7195, 7773, 7789, 8565, 9015, 9631, 9634])